How well do Chemical genetic interactions predict drug classes? Here, the cleaned data generated is used to train and test a logistic regression model. Testing is accomplished by leaving one drug out in each iteration

In [7]:
import itertools
import pathlib
import re
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
import xlsxwriter
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

## Functions

In [8]:
def get_unique_drugs(drug_list):
    """returns unique drugs maintaining order"""
    out = []
    for drug in drug_list:
        if drug not in out:
            out.append(drug)
    return out

In [9]:
def collapse_confusion(confusion, drug_class, dir_name):
    """takes a confusion matrix of size [ndrugs, nclasses] and collapses it to make [nclasses, nclasses]. Saves the result as a figure"""
    name=os.path.basename(dir_name)
    drug_class = drug_class.set_index('Drug')
    confusion = confusion.join(drug_class, how='inner')
    confusion_classes = confusion.groupby('Class').mean()
    heat_c = sns.heatmap(confusion_classes, annot=True, cmap="YlGnBu", fmt='1.2f', square=True)
    heat_c.set(xlabel='Predicted Class', ylabel='True Class', title=name)
    fig_c = heat_c.get_figure()
    
    fig_path = pathlib.Path.cwd().joinpath(dir_name, name+'_conf_pp_collapsed.png')
    fig_c.savefig(fig_path, bbox_inches='tight', dpi=300)    
    plt.close()

In [10]:
# def make_coef_df(coef_df, strain_list, y, dir_name):
#     """Takes in a matrix of weights of size [nclasses, nstrains] and saves a transposed and annotated version with abs values"""
#     name=os.path.basename(dir_name)
#     coef_df = coef_df.transpose()
#     coef_df.columns = sorted(list(set(y)))
#     coef_df.index = strain_list
#     coef_df_abs = coef_df.abs()
#     pathlib.Path.cwd().joinpath(dir_name, name+'.png')
#     coef_df.to_csv(pathlib.Path.cwd().joinpath(dir_name, name+'_coefs.csv'))
#     coef_df_abs.to_csv(pathlib.Path.cwd().joinpath(dir_name, name+'_coefs_abs.csv'))


In [31]:
def fit_model(X, y, drug_list, correct_imbal):
    """Implements XGBoost with CV. correct_imbal can be None or SMOTE"""
    
    print ("Starting predictions")
    
    classes = sorted(list(set(y)))
    indexes = np.arange(len(y))
    unique_drugs = get_unique_drugs(drug_list)
    confusion = np.empty((len(unique_drugs),len(classes)))
    var_importances = np.zeros(X.shape[1])
    
    #Loop
    for j,drug in enumerate(unique_drugs):
        print(drug, end=',')
        #Get train and test indices
        train_ind = indexes[drug_list != drug]
        test_ind = indexes[drug_list == drug]
        
        #Correct imbalance
        if correct_imbal == 'SMOTE':
            X_train, y_train= SMOTE().fit_sample(X[train_ind], y[train_ind])
        elif correct_imbal == None:
            X_train,y_train=(X[train_ind], y[train_ind]) 
            
        #fit    
        model = XGBClassifier(objective='multi:softmax', max_depth=1, verbosity=1, tree_method='hist', learning_rate=0.9)
        model.fit(X_train, y_train)
        
        #add the weights to a dataframe.These get added every iteration
        var_importances += model.feature_importances_      
        #calculate predict probabilities for each test. Yields matrix of size [ntests, nclasses]
        confusion_drug = model.predict_proba(X[test_ind])
        #collapse this matrix by calculating mean over all replicates. Yields a matrix of size[1,nclasses]. Append to confusion
        confusion[j] = confusion_drug.mean(axis=0)
        
    confusion_df = pd.DataFrame(confusion, columns=classes, index=unique_drugs)
    var_importances = var_importances/len(unique_drugs)
    print("Predictions finished")
    
    return confusion_df, var_importances
   

In [28]:
def do_XgBoost(df, dir_name, drug_class, correct_imbal=None):
    """Prepares input data and implements RF. Uses '.' to look for columns for X. Requires colnames 'Class' and 'Drugs'"""
    X = df.filter(regex=('\.')).values
    y = df['Class'].values
    drug_list = np.array(df['Drug'])
    print(X.shape, len(y))
    
    #Create directory for results
    try:
        pathlib.Path.cwd().joinpath(dir_name).mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        pass
    
    #Run
    confusion, coef = fit_model(X, y, drug_list, correct_imbal)
    
    #Generate a csv file with the coefficients of the logistic regression. This allows identification of possible important predictors of the model.
#     make_coef_df(coef, df.filter(regex=('\.')).columns, y, dir_name)
    #Generates a confusion matrix
    collapse_confusion(confusion, drug_class, dir_name)
    return None 

## HypoIII

In [23]:
dir_name = './Analysis/9-Logistic_regression/HypoIII/XgBoost'

In [14]:
hypoIII = pd.read_csv(pathlib.Path.cwd().joinpath('Analysis','8-Clean_for_ML', 'HypoIII_for_ML_encoded.csv'))

In [15]:
hypoIII.head()

,Drug,Conc,Replicate,strain.0,strain.1,strain.10,strain.100,strain.101,strain.102,strain.103,strain.104,strain.105,strain.106,strain.107,strain.108,strain.109,strain.11,strain.110,strain.111,strain.112,strain.113,strain.114,strain.115,strain.116,strain.117,strain.118,strain.119,strain.12,strain.120,strain.121,strain.122,strain.123,strain.124,strain.125,strain.126,strain.127,strain.128,strain.129,strain.13,strain.130,strain.131,strain.132,strain.133,strain.134,strain.135,strain.136,strain.137,strain.138,strain.139,strain.14,strain.140,strain.141,strain.142,strain.143,strain.144,strain.145,strain.146,strain.147,strain.148,strain.149,...,strain.46,strain.47,strain.48,strain.49,strain.5,strain.50,strain.51,strain.52,strain.53,strain.54,strain.55,strain.56,strain.57,strain.58,strain.59,strain.6,strain.60,strain.61,strain.62,strain.63,strain.64,strain.65,strain.66,strain.67,strain.68,strain.69,strain.7,strain.70,strain.71,strain.72,strain.73,strain.74,strain.75,strain.76,strain.77,strain.78,strain.79,strain.8,strain.80,strain.81,strain.82,strain.83,strain.84,strain.85,strain.86,strain.87,strain.88,strain.89,strain.9,strain.90,strain.91,strain.92,strain.93,strain.94,strain.95,strain.96,strain.97,strain.98,strain.99,Class
0,drug_0,2xMIC,1,0.359612,-0.911401,1.554297,0.069421,-2.972960,1.185445,0.421721,1.260755,0.359771,-0.216278,-0.810467,0.207221,-1.367722,0.580652,-1.820107,-1.451017,-3.426601,-3.751756,0.392810,0.498581,1.411772,1.724880,0.524027,1.138232,1.152429,1.166336,0.387385,1.095914,0.218686,1.133761,1.445079,0.294032,1.048346,-0.374432,0.476607,1.215083,0.997137,1.177133,1.308518,1.238719,1.658974,0.654698,0.470250,0.942999,0.073280,0.238394,1.062685,1.247293,1.114485,0.578855,1.381173,0.187340,0.066104,0.880239,0.680201,0.053263,1.240228,...,-0.480681,0.815519,0.700967,0.453607,0.609096,0.505015,0.810550,-0.276599,0.646931,1.394900,1.566079,1.169090,1.520940,1.543827,0.980045,1.105089,0.320256,0.954399,0.868444,1.176105,-3.318928,1.159871,-1.226029,-0.158422,-2.684306,-1.050331,1.189915,0.298784,0.639840,-2.709869,-0.741713,-0.728370,-0.187352,1.824396,1.112346,-0.253682,-0.673390,1.375321,-0.457863,0.398109,0.592303,-0.799946,0.840703,0.070947,-0.579263,-0.133004,-0.615987,-1.302696,0.021997,0.621211,0.176432,0.449538,0.131855,-0.794105,1.381503,-0.938048,-2.939168,1.492050,-0.387570,Class 4
1,drug_0,2xMIC,2,0.922680,0.156397,1.587909,0.587814,0.183369,0.565838,1.133654,1.458857,1.060058,0.652811,0.243900,0.676625,0.337484,1.324324,0.177894,-0.463206,0.715229,0.805111,0.585464,1.027006,1.146962,1.520532,0.661632,1.374707,1.150593,1.255008,1.020693,1.517711,0.136543,1.130481,1.161470,0.414035,0.815587,-0.382349,0.667372,1.555939,1.206262,1.342124,1.224571,1.329476,1.206735,0.959326,0.631543,1.081797,0.669234,0.527241,1.116695,1.365894,1.049876,0.812022,1.781029,0.271998,0.105962,1.319814,1.201911,0.407598,1.359833,...,0.336449,1.305055,0.983897,0.751580,0.562391,1.087239,0.699624,-0.724554,1.711397,1.303371,1.985980,1.175662,1.700578,1.617191,1.374513,1.201260,1.246523,1.081021,1.488550,1.144523,-1.373495,1.057028,0.218779,1.126913,-1.112535,0.133982,1.241503,1.262696,1.216395,0.631269,0.004816,1.344122,0.310338,1.951353,1.460946,0.234290,0.788589,1.255002,0.099004,0.351235,1.460158,-0.311394,0.641070,1.230159,-0.025233,0.359026,0.335645,0.820854,0.585121,0.624340,0.702362,0.783339,0.747126,0.113741,1.673887,-0.256643,-0.483839,1.684535,1.228144,Class 4
2,drug_0,2xMIC,3,1.003276,0.171781,1.870643,0.381407,-0.678374,1.030591,1.469478,1.682974,1.216445,0.090234,0.143118,0.347489,0.637329,1.513611,-0.080517,-0.265340,-3.230579,-3.403731,0.756657,0.841526,1.650220,1.569363,0.684088,1.482705,1.092077,1.600532,0.603044,1.408525,0.774618,1.592050,1.673931,0.103614,1.251104,0.069059,0.850141,1.681501,1.197150,1.508897,1.209124,1.346898,1.112427,0.760226,0.980105,0.989179,0.794967,1.247128,1.687463,1.369269,1.350816,0.701479,1.520240,0.499148,0.089200,1.140728,1.524682,0.570564,1.416926,...,0.680274,1.826017,1.307428,0.630365,0.7

In [16]:
hypoIII['Class'].unique()

array(['Class 4', 'Class 1', 'Class 6', nan, 'Class 5', 'Class 3',
       'Class 2'], dtype=object)

In [17]:
hypoIII = hypoIII[~hypoIII['Class'].isna()]
hypoIII = hypoIII.sort_values(by=['Class', 'Drug'])
hypoIII.head()

,Drug,Conc,Replicate,strain.0,strain.1,strain.10,strain.100,strain.101,strain.102,strain.103,strain.104,strain.105,strain.106,strain.107,strain.108,strain.109,strain.11,strain.110,strain.111,strain.112,strain.113,strain.114,strain.115,strain.116,strain.117,strain.118,strain.119,strain.12,strain.120,strain.121,strain.122,strain.123,strain.124,strain.125,strain.126,strain.127,strain.128,strain.129,strain.13,strain.130,strain.131,strain.132,strain.133,strain.134,strain.135,strain.136,strain.137,strain.138,strain.139,strain.14,strain.140,strain.141,strain.142,strain.143,strain.144,strain.145,strain.146,strain.147,strain.148,strain.149,...,strain.46,strain.47,strain.48,strain.49,strain.5,strain.50,strain.51,strain.52,strain.53,strain.54,strain.55,strain.56,strain.57,strain.58,strain.59,strain.6,strain.60,strain.61,strain.62,strain.63,strain.64,strain.65,strain.66,strain.67,strain.68,strain.69,strain.7,strain.70,strain.71,strain.72,strain.73,strain.74,strain.75,strain.76,strain.77,strain.78,strain.79,strain.8,strain.80,strain.81,strain.82,strain.83,strain.84,strain.85,strain.86,strain.87,strain.88,strain.89,strain.9,strain.90,strain.91,strain.92,strain.93,strain.94,strain.95,strain.96,strain.97,strain.98,strain.99,Class
32,drug_12,1xMIC,1,0.157042,-0.266909,0.778431,-0.742298,-1.403845,-0.119274,-0.121111,-0.718754,-0.280492,-1.602764,-1.126554,-1.149551,-0.473100,0.542094,-0.785173,-1.412742,-2.096643,-3.171623,0.827943,0.378658,0.641896,1.124993,-0.007903,0.298635,0.447871,0.926045,0.053191,0.762616,-0.870729,0.810938,0.671531,-0.925063,0.701664,-0.779141,0.341597,0.252021,0.211068,0.266202,0.727074,0.880620,0.518669,0.975789,1.216206,0.419382,0.348473,0.653892,0.499445,0.066409,0.567816,0.219156,0.475107,0.537722,0.280663,0.632458,0.559289,-0.868800,0.996530,...,-0.448777,0.472112,0.517902,1.157424,0.189990,0.230775,0.154338,-1.165690,0.381459,0.199641,0.541681,0.204567,0.884418,0.967522,0.755534,0.802701,-0.615761,-0.230246,-0.429073,0.314987,-0.560306,0.683733,-1.079716,-0.913744,-1.722753,-1.749246,0.057959,0.429104,0.343316,-1.448757,-0.702150,-0.454103,-0.460843,0.602093,0.063982,-0.357222,-0.442916,0.812060,-1.194630,0.414144,0.198268,-0.534788,0.640802,0.218574,-0.470746,-0.262892,-1.111172,0.003111,0.643424,-0.833646,0.248958,0.932941,0.607170,-0.821963,0.601467,-0.789908,-1.460669,0.122685,0.123163,Class 1
33,drug_12,1xMIC,2,0.708403,0.006475,1.139620,-0.038633,-0.329288,0.379601,0.358161,-0.264607,0.510209,-0.883644,-0.302873,0.283763,0.091318,1.042446,-0.127203,-0.626009,-1.744943,-2.364243,1.059162,0.671607,1.004307,1.482232,0.386582,0.977409,0.717483,1.373455,0.776302,1.339948,-0.232408,1.193438,0.969456,-0.461449,1.231046,-0.439778,0.609038,0.931643,0.691971,0.876108,1.123780,1.069233,0.714467,1.271274,1.289706,0.937329,1.160850,1.280910,0.660363,0.600234,1.418871,0.556696,1.112562,1.596490,0.783376,1.404901,1.189046,-0.373431,1.280858,...,-0.159533,1.014590,0.898100,1.746548,0.800901,0.774466,0.510358,-0.448643,1.034559,0.603356,1.123500,0.461917,1.299424,1.133355,1.173415,0.939637,-0.098450,0.140549,-0.024704,0.532537,-0.954561,0.839826,-0.257837,0.456750,-1.023725,-0.796095,0.611944,0.893918,0.895639,-2.363843,-0.078162,-0.234343,0.026609,1.038041,0.519249,-0.025000,0.832512,1.035212,-1.278804,1.227889,0.669623,0.290480,1.141739,0.711815,0.021742,0.120114,-0.113922,0.391222,0.748798,-0.322230,0.603231,0.170081,1.121570,0.065348,0.884810,-0.419019,-1.035201,0.685974,-0.275755,Class 1
34,drug_12,1xMIC,3,0.398040,0.012963,1.160000,-0.418280,-1.001709,0.449131,0.061164,-0.546433,0.324482,-1.756132,-0.589660,-1.365794,0.091419,1.105662,-0.504009,-1.002100,-1.657535,-2.539870,1.236005,0.732922,1.039564,1.302333,0.340896,0.750694,0.518211,1.160990,0.487908,1.163658,-0.451576,1.115584,0.687355,-0.323057,0.990950,-0.942850,0.968652,0.449410,0.845554,0.784458,1.074155,1.082871,0.752688,1.214473,1.216790,0.937082,0.833683,1.157189,0.854792,0.342527,1.108039,0.414539,1.051447,0.767942,0.804557,0.983331,0.912793,-0.478254,1.2

In [18]:
hypoIII['Class'].value_counts()

Class 1    527
Class 4    143
Class 5    125
Class 6    115
Class 3     56
Class 2     32
Name: Class, dtype: int64

In [19]:
drug_class = hypoIII[['Drug', 'Class']].drop_duplicates()
drug_class.head()

,Drug,Class
32,drug_12,Class 1
49,drug_13,Class 1
53,drug_14,Class 1
94,drug_2,Class 1
108,drug_20,Class 1


In [32]:
do_XgBoost(hypoIII, dir_name, drug_class=drug_class, correct_imbal='SMOTE')

(998, 425) 998
Starting predictions
drug_12,drug_13,drug_14,drug_2,drug_20,drug_21,drug_22,drug_24,drug_25,drug_26,drug_28,drug_29,drug_31,drug_32,drug_33,drug_34,drug_41,drug_42,drug_47,drug_48,drug_49,drug_50,drug_51,drug_59,drug_63,drug_64,drug_65,drug_67,drug_68,drug_71,drug_73,drug_77,drug_8,drug_81,drug_82,drug_83,drug_85,drug_70,drug_78,drug_46,drug_53,drug_62,drug_69,drug_0,drug_1,drug_30,drug_4,drug_55,drug_56,drug_57,drug_58,drug_61,drug_66,drug_84,drug_19,drug_3,drug_36,drug_37,drug_38,drug_44,drug_74,drug_75,drug_76,drug_17,drug_39,drug_40,drug_45,drug_5,drug_52,drug_54,drug_72,drug_79,drug_80,drug_9,Predictions finished
